# Analisis Sentimen

In [1]:
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv("dataset/raw_dataset/pilpres.csv", on_bad_lines='skip')
df.head()

,Unnamed: 0.1,Unnamed: 0,created_at,id_str,full_text,quote_count,reply_count,retweet_count,favorite_count,lang,user_id_str,conversation_id_str,username,tweet_url
0,0,0,Fri Dec 15 04:04:44 +0000 2023,1.735511e+18,"@dedy_pram Pastilah, Prabowo Anies nggak punya...",0.0,0,0.0,0.0,in,797761417894051840,1.735147e+18,MarahIchsan,https://twitter.com/MarahIchsan/status/1735511...
1,1,1,Fri Dec 15 04:04:33 +0000 2023,1.735511e+18,Suasana pedesaan memang selalu menyenangkan......,0.0,0,0.0,0.0,in,936445409605459968,1.735511e+18,Agung_wiyo,https://twitter.com/Agung_wiyo/status/17355111...
2,2,2,Fri Dec 15 04:04:31 +0000 2023,1.735511e+18,Program anies hanyalah mengubah nama rumah sak...,0.0,0,0.0,0.0,in,768652236,1.735511e+18,Alfakton_i,https://twitter.com/Alfakton_i/status/17355111...
3,3,3,Fri Dec 15 04:04:27 +0000 2023,1.735511e+18,@kumparan @aniesbaswedan anies... BERBOHONG DA...,0.0,0,0.0,0.0,in,1710823909275410432,1.735146e+18,justtheway333,https://twitter.com/justtheway333/status/17355...
4,4,4,Fri Dec 15 04:04:26 +0000 2023,1.735511e+18,Tidak Bicara Sesuai Fakta! Anies Dengan Lantan...,0.0,0,1.0,1.0,in,1661757337554784264,1.735511e+18,singgihabdrchmn,https://twitter.com/singgihabdrchmn/status/173...


In [3]:
df = df[['full_text','username', 'created_at', 'tweet_url']]
df

,full_text,username,created_at,tweet_url
0,"@dedy_pram Pastilah, Prabowo Anies nggak punya...",MarahIchsan,Fri Dec 15 04:04:44 +0000 2023,https://twitter.com/MarahIchsan/status/1735511...
1,Suasana pedesaan memang selalu menyenangkan......,Agung_wiyo,Fri Dec 15 04:04:33 +0000 2023,https://twitter.com/Agung_wiyo/status/17355111...
2,Program anies hanyalah mengubah nama rumah sak...,Alfakton_i,Fri Dec 15 04:04:31 +0000 2023,https://twitter.com/Alfakton_i/status/17355111...
3,@kumparan @aniesbaswedan anies... BERBOHONG DA...,justtheway333,Fri Dec 15 04:04:27 +0000 2023,https://twitter.com/justtheway333/status/17355...
4,Tidak Bicara Sesuai Fakta! Anies Dengan Lantan...,singgihabdrchmn,Fri Dec 15 04:04:26 +0000 2023,https://twitter.com/singgihabdrchmn/status/173...
...,...,...,...,...
2599,"@kompascom Memang tidak mudah, sekarang masyar...",HAPOSANM,Fri Dec 15 03:56:02 +0000 2023,https://twitter.com/HAPOSANM/status/1735508992...
2600,Visi Prabowo - Gibran membawa harapan baru bag...,istimnatiara,Fri Dec 15 03:56:01 +0000 2023,https://twitter.com/istimnatiara/status/173550...
2601,@supersaiyaindo @ch_chotimah2 Pembenaran akan ...,DylanBr03197352,Fri Dec 15 03:56:01 +0000 2023,https://twitter.com/DylanBr03197352/status/173...
2602,Prabowo Gibran berkomitmen untuk mengoptimalka...,clara144495,Fri Dec 15 03:56:00 +0000 2023,https://twitter.com/clara144495/status/1735508...


## Data Cleaning

Membersihkan data dari data yang duplikat maupun data yang bernilai NULL

In [4]:
df.shape

(2604, 4)

In [5]:
df = df.drop_duplicates(subset=['full_text'])

In [6]:
df.duplicated().sum()

0

In [7]:
df = df.dropna()

In [8]:
df.isnull().sum()

full_text     0
username      0
created_at    0
tweet_url     0
dtype: int64

In [9]:
df.shape

(2557, 4)

### Tweet Cleaning

Menghilangkan simbol @, mention, retweet, dan karakter yang tidak perlukan

In [10]:
def clean_tweet(text):
    text = re.sub(r'@[A-Za-z0-9_]+', ' ', text)
    text = re.sub(r'#\w+','', text)
    text = re.sub(r'RT[\s]+', '', text)
    text = re.sub(r'https?://\S+', '', text)

    text = re.sub(r'[^A-Za-z0-9 ]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

df['full_text'] = df['full_text'].apply(clean_tweet)
df['full_text'] = df['full_text'].str.lower()

In [11]:
selling = ['shopee', 'cod']
df = df.drop(df[df['full_text'].str.contains('|'.join(selling))].index)
df

,full_text,username,created_at,tweet_url
0,pastilah prabowo anies nggak punya prestasi,MarahIchsan,Fri Dec 15 04:04:44 +0000 2023,https://twitter.com/MarahIchsan/status/1735511...
1,suasana pedesaan memang selalu menyenangkan ud...,Agung_wiyo,Fri Dec 15 04:04:33 +0000 2023,https://twitter.com/Agung_wiyo/status/17355111...
2,program anies hanyalah mengubah nama rumah sak...,Alfakton_i,Fri Dec 15 04:04:31 +0000 2023,https://twitter.com/Alfakton_i/status/17355111...
3,anies berbohong dan nipu tanpa rasa bersalah,justtheway333,Fri Dec 15 04:04:27 +0000 2023,https://twitter.com/justtheway333/status/17355...
4,tidak bicara sesuai fakta anies dengan lantang...,singgihabdrchmn,Fri Dec 15 04:04:26 +0000 2023,https://twitter.com/singgihabdrchmn/status/173...
...,...,...,...,...
2599,memang tidak mudah sekarang masyarakat mau tau...,HAPOSANM,Fri Dec 15 03:56:02 +0000 2023,https://twitter.com/HAPOSANM/status/1735508992...
2600,visi prabowo gibran membawa harapan baru bagi ...,istimnatiara,Fri Dec 15 03:56:01 +0000 2023,https://twitter.com/istimnatiara/status/173550...
2601,pembenaran akan selalu ada tapi jika anda mau ...,DylanBr03197352,Fri Dec 15 03:56:01 +0000 2023,https://twitter.com/DylanBr03197352/status/173...
2602,prabowo gibran berkomitmen untuk mengoptimalka...,clara144495,Fri Dec 15 03:56:00 +0000 2023,https://twitter.com/clara144495/status/1735508...


## Processing

Merupakan rangkaian beberapa proses manipulasi data sebelum data digunakan untuk membangun model machine learning. Beberapa proses yang dilakukan seperti:
1. Normalisasi
2. Stopword
3. Tokenisasi
4. Stemming

### Normalisasi

Sebuah proses menormalisasi setiap kata yang memiliki imbuhan, typo, atau yang berupa singkatan menjadi kata dasarnya atau kata baku

In [12]:
# Normalisasi
from normalize_words import norm

def normalisasi(str_text):
    for i in norm:
        str_text = str_text.replace(i, norm[i])
    return str_text
    
df['full_text'] = df['full_text'].apply(lambda x: normalisasi(x))
df

,full_text,username,created_at,tweet_url
0,pastilah tidak punya prestasi,MarahIchsan,Fri Dec 15 04:04:44 +0000 2023,https://twitter.com/MarahIchsan/status/1735511...
1,suasana pedesaan memang selalu menyenangkan ud...,Agung_wiyo,Fri Dec 15 04:04:33 +0000 2023,https://twitter.com/Agung_wiyo/status/17355111...
2,program hanyalah mengubah nama rumah sakit se ...,Alfakton_i,Fri Dec 15 04:04:31 +0000 2023,https://twitter.com/Alfakton_i/status/17355111...
3,berbohong dan tipu tanpa rasa bersalah,justtheway333,Fri Dec 15 04:04:27 +0000 2023,https://twitter.com/justtheway333/status/17355...
4,tidak bicara sesuai fakta dengan lantang sebut...,singgihabdrchmn,Fri Dec 15 04:04:26 +0000 2023,https://twitter.com/singgihabdrchmn/status/173...
...,...,...,...,...
2599,memang tidak mudah sekarang masyarakat mau tau...,HAPOSANM,Fri Dec 15 03:56:02 +0000 2023,https://twitter.com/HAPOSANM/status/1735508992...
2600,visi membawa harapan baru bagi indonesia dalam...,istimnatiara,Fri Dec 15 03:56:01 +0000 2023,https://twitter.com/istimnatiara/status/173550...
2601,pembenaran akan selalu ada tapi jika anda mau ...,DylanBr03197352,Fri Dec 15 03:56:01 +0000 2023,https://twitter.com/DylanBr03197352/status/173...
2602,berkomitmen untuk mengoptimalkan potensi indu...,clara144495,Fri Dec 15 03:56:00 +0000 2023,https://twitter.com/clara144495/status/1735508...


### Stopword

Proses menghilangkan stopwords dari data yang biasanya memiliki imbuhan atau kata yang tidak memiliki makna yang berpengaruh pada proses membangun model machine learning

In [13]:
# Stopwords

import Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
# more_stop_words = ['tidak']

stop_words = StopWordRemoverFactory().get_stop_words()
# stop_words.extend(more_stop_words)

new_array = ArrayDictionary(stop_words)
stop_words_remover_new = StopWordRemover(new_array)

def stopword(str_text):
    str_text = stop_words_remover_new.remove(str_text)
    return str_text

df['full_text'] = df['full_text'].apply(lambda x: stopword(x))
df.head()

,full_text,username,created_at,tweet_url
0,pastilah punya prestasi,MarahIchsan,Fri Dec 15 04:04:44 +0000 2023,https://twitter.com/MarahIchsan/status/1735511...
1,suasana pedesaan memang selalu menyenangkan ud...,Agung_wiyo,Fri Dec 15 04:04:33 +0000 2023,https://twitter.com/Agung_wiyo/status/17355111...
2,program hanyalah mengubah nama rumah sakit se ...,Alfakton_i,Fri Dec 15 04:04:31 +0000 2023,https://twitter.com/Alfakton_i/status/17355111...
3,berbohong tipu rasa bersalah,justtheway333,Fri Dec 15 04:04:27 +0000 2023,https://twitter.com/justtheway333/status/17355...
4,bicara sesuai fakta lantang sebut lampung memp...,singgihabdrchmn,Fri Dec 15 04:04:26 +0000 2023,https://twitter.com/singgihabdrchmn/status/173...


### Tokenizing

Atau tokenisasi kalimat menjadi potongan-potongan kata kedalam array/list yang memiliki token sebagai identitas yang dapat membedakan sebuah kata dengan kata lainnya

In [14]:
# Tokenisasi

tokenized = df['full_text'].apply(lambda x:x.split())
tokenized

0                             [pastilah, punya, prestasi]
1       [suasana, pedesaan, memang, selalu, menyenangk...
2       [program, hanyalah, mengubah, nama, rumah, sak...
3                       [berbohong, tipu, rasa, bersalah]
4       [bicara, sesuai, fakta, lantang, sebut, lampun...
                              ...                        
2599    [memang, mudah, sekarang, masyarakat, mau, tau...
2600    [visi, membawa, harapan, baru, indonesia, menc...
2601    [pembenaran, selalu, tapi, mau, merunut, kata,...
2602    [berkomitmen, mengoptimalkan, potensi, industr...
2603    [program, program, jembatan, menuju, kemajuan,...
Name: full_text, Length: 2555, dtype: object

### Stemming 

Proses mengganti dan mencari kata dasar dari sebuah kata

In [ ]:
# Stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming(text_cleaning):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in text_cleaning:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean = []
    d_clean = " ".join(do)
    print(d_clean)
    return d_clean

tokenized = tokenized.apply(stemming)
# tokenized.to_csv('dataset/processing_data.csv', index=False)

In [17]:
# Simpan hasil processing
data = pd.read_csv('dataset/processing_data.csv', encoding='latin1')
data.head()

,full_text
0,pasti punya prestasi
1,suasana desa memang selalu senang udara segar ...
2,program hanya ubah nama rumah sakit se indones...
3,bohong tipu rasa salah
4,bicara sesuai fakta lantang sebut lampung puny...


## Labeling

Memberi label terhadap setiap tweet yang akan digunakan untuk melatih model machine learning. Terdapat 3 label yang digunakan yaitu positif, negatif, dan netral.

### Translate

Menerjemahkan data kedalam bahasa inggris agar memudahkan penggunaan library Translate untuk labeling

In [ ]:
from translate import Translator

def convert_eng(tweet):
    translator = Translator(to_lang="en", from_lang="id")
    translation = translator.translate(tweet)
    return translation

data['tweet_eng'] = data['full_text'].apply(convert_eng)
data.to_csv('dataset/translated_data.csv')

### Proses Labeling

Proses labeling menggunakan library textblob

In [13]:
import pandas as pd
dataset = pd.read_csv('dataset/translated_data.csv', index_col=0)
dataset

,full_text,tweet_english
0,eh botak bilang capres lu dulu ngobrol ama ani...,uh bald said your presidential candidate used ...
1,menhan prabowo subianto abdi korban prajurit t...,minister of defense prabowo subianto servant o...
2,elektabilitas tembus 45 persen ahy prabowo men...,electability penetrates 45 percent ahy prabowo...
3,ahy demokrat siap kawal menang prabowo gibran ...,ahy democrat ready to control winning prabowo ...
4,kelola lingkung lanjut komitmen prabowo gibran...,manage the environment further commitment prab...
...,...,...
302,sama pak mas silaturahmi kyai ulama amp santri...,with Mr. Kyai ulama & santri ponpes miftahul h...
303,sdhlah sdh lalu bukti makan ego sendiri pak pr...,sdhlah sdh then evidence of eating his own ego...
304,elektabilitas prabowo gibran selalu unggul ahy...,electability prabowo gibran always superior ah...
305,jir striker prabowo,jir striker prabowo


In [18]:
import preprocessor as p
from textblob import TextBlob
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
data_tweet = list(dataset['tweet_english'])
polaritas = 0

status = []
total_positif = total_negatif = total_netral = total = 0

for i, tweet in enumerate(data_tweet):
    analysis = TextBlob(tweet)
    polaritas += analysis.polarity

    if analysis.sentiment.polarity > 0.0:
        total_positif += 1
        status.append('Positif')
    elif analysis.sentiment.polarity == 0.0:
        total_netral += 1
        status.append('Netral')
    else:
        total_negatif += 1
        status.append('Negatif')

    total += 1

print(f'Hasil Analisis Data:\nPositif = {total_positif}\nNetral = {total_netral}\nNegatif = {total_negatif}\n\nTotal Data = {total}')

Hasil Analisis Data:
Positif = 156
Netral = 102
Negatif = 49

Total Data = 307


In [16]:
dataset['klasifikasi'] = status
dataset

,full_text,tweet_english,klasifikasi
0,eh botak bilang capres lu dulu ngobrol ama ani...,uh bald said your presidential candidate used ...,Negatif
1,menhan prabowo subianto abdi korban prajurit t...,minister of defense prabowo subianto servant o...,Negatif
2,elektabilitas tembus 45 persen ahy prabowo men...,electability penetrates 45 percent ahy prabowo...,Positif
3,ahy demokrat siap kawal menang prabowo gibran ...,ahy democrat ready to control winning prabowo ...,Positif
4,kelola lingkung lanjut komitmen prabowo gibran...,manage the environment further commitment prab...,Positif
...,...,...,...
302,sama pak mas silaturahmi kyai ulama amp santri...,with Mr. Kyai ulama & santri ponpes miftahul h...,Netral
303,sdhlah sdh lalu bukti makan ego sendiri pak pr...,sdhlah sdh then evidence of eating his own ego...,Positif
304,elektabilitas prabowo gibran selalu unggul ahy...,electability prabowo gibran always superior ah...,Positif
305,jir striker prabowo,jir striker prabowo,Netral


# Klasifikasi Naive Bayes

Membuat model machine learning menggunakan algoritma naive bayes untuk mengklasifikasi sentimen

In [17]:
import pandas as pd
# data_set = pd.read_csv('dataset/translated_ready.csv', index_col=0)
# data_set

data_set = dataset.drop(['tweet_english'], axis=1, inplace=False)
data_set = [tuple(x) for x in data_set.to_records(index=False)]

### Data Split

Membagi dataset menjadi training set untuk melatih model ML Naive Bayes

In [18]:
import random

set_positif = []
set_negatif = []
set_netral = []

for n in data_set:
    if(n[1] == 'Positif'):
        set_positif.append(n)
    elif(n[1] == 'Negatif'):
        set_negatif.append(n)
    else:
        set_netral.append(n)

set_positif = random.sample(set_positif, k=int(len(set_positif)/2))
set_negatif = random.sample(set_negatif, k=int(len(set_negatif)/2))
set_netral = random.sample(set_netral, k=int(len(set_netral)/2))

train = set_positif+set_negatif+set_netral

train_set = []
for n in train:
    train_set.append(n)
    

### Training

Melatih model Naive Bayes menggunakan modul NaiveBayesClassifier

In [19]:
from textblob.classifiers import NaiveBayesClassifier

cl = NaiveBayesClassifier(train_set)
print("Akurasi Test : ",cl.accuracy(data_set))

Akurasi Test :  0.749185667752443


In [9]:
data_tweetNB = list(dataset['tweet_eng'])
polaritasNB = 0

statusNB = []
total_positif = total_negatif = total_netral = total = 0

for i, tweet in enumerate(data_tweetNB):
    analysis = TextBlob(tweet, classifier=cl)

    if analysis.classify() == 'Positif':
        total_positif += 1
    elif analysis.classify() == 'Netral':
        total_netral += 1
    else:
        total_negatif += 1

    statusNB.append(analysis.classify())
    total+=1

print(f'Hasil Analisis Data:\nPositif = {total_positif}\nNetral = {total_netral}\nNegatif = {total_negatif}\n\nTotal Data = {total}')

Hasil Analisis Data:
Positif = 2182
Netral = 349
Negatif = 24

Total Data = 2555


### Save Model
Simpan Model yang telah di Training

In [17]:
import pickle

f = open('models/NBPilpres_baru.pickle', 'wb')
pickle.dump(cl, f)
f.close

<function BufferedWriter.close>

In [20]:
# Load Model yang disimpan
loaded_model = pickle.load(open('models/NBPilpres_baru.pickle', 'rb'))

In [25]:
# Testing Klasifikasi Model Naive Bayes

test = pd.read_csv('translated_dataset/TerjemahanAnies.csv', index_col=0)
data_tweetNB = list(test['full_text'])
polaritasNB = 0

statusNB = []
total_positif = total_negatif = total_netral = total = 0

for i, tweet in enumerate(data_tweetNB):
    analysis = TextBlob(tweet, classifier=loaded_model)

    if analysis.classify() == 'Positif':
        total_positif += 1
    elif analysis.classify() == 'Netral':
        total_netral += 1
    else:
        total_negatif += 1

    statusNB.append(analysis.classify())
    total+=1

print(f'Hasil Analisis Data:\nPositif = {total_positif}\nNetral = {total_netral}\nNegatif = {total_negatif}\n\nTotal Data = {total}')

Hasil Analisis Data:
Positif = 181
Netral = 94
Negatif = 3

Total Data = 278
